In [85]:
import numpy as np
from header import *
from nsp.utils.base_conv import *
import argparse
import sys
sys.path.append('../nsp')
from nsp.utils.func import *

In [91]:
Sz = np.zeros([3,3])
Sz[0,0] = 1
Sz[2,2] = -1
Sx = np.zeros([3, 3])
Sx[1,0] = Sx[0,1] = Sx[2,1] = Sx[1,2] = 1/np.sqrt(2)
Sy = np.zeros([3, 3], dtype=np.complex64)
Sy[1,0] = Sy[2,1] = 1j/np.sqrt(2)
Sy[0,1] = Sy[1,2] = -1j/np.sqrt(2)


SzSz = np.kron(Sz,Sz).astype(np.float64)
SxSx = np.kron(Sx,Sx).astype(np.float64)
SySy = np.kron(Sy,Sy).astype(np.float64)

lh = SzSz + SxSx + SySy
J = [1, 1/3]
lh = lh + lh@lh*J[1]
lh = -lh

/tmp/ipykernel_13678/1961612155.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  SySy = np.kron(Sy,Sy).astype(np.float64)


In [93]:
from random import randint

D = 3
seed = randint(0, 2<<32 - 1)
loss_mes = nsp.loss.MES(lh, [D, D], pout = False)
model = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
solver = UnitaryTransTs(RiemanUnitaryCG,model, loss_mes, lr = 0.005, momentum=0.1, af = False)
ret = solver.run(3000, False)
# if loss_name == "mes":
#     solver = UnitaryTransTs(RiemanUnitaryCG, model, loss_mes, lr = 0.005, momentum=0.1, af = False)
#     ret = solver.run(1000, False)
print(f"res = {ret.fun} / seed = {seed}")
best_model = ret.model
best_loss = loss_mes([ret.model.matrix()]*loss_mes._n_unitaries).detach().numpy()
print(f"loss(mes) : {best_loss}")

target loss      : 2.0000000530
initial loss     : 2.8322625136
loss upper bound : 2.4880338969




  0%|          | 0/3000 [00:00<?, ?it/s]

/home/user/project/python/nsp/test/../nsp/optim/rieman_unitary_optim.py:229: UserWarning: An output with one or more elements was resized since it had shape [1, 3, 3], which does not match the required output shape [1, 1, 3, 3]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  return loss(torch.matrix_exp(-t*H)@U).item()
/home/user/.local/lib/python3.9/site-packages/torch/autograd/__init__.py:276: UserWarning: An output with one or more elements was resized since it had shape [1, 6, 6], which does not match the required output shape [1, 1, 6, 6]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements w

No local minimum found
8.018593175384687e-10 6.208908170587945e-23 2.0000000529811763 2.000000052981176
No local minimum found
2.435772181012275e-10 1.5661182203516313e-22 2.000000052981176 2.000000052981176
No local minimum found
1.544770754683377e-10 6.394700849572665e-23 2.000000052981176 2.0000000529811754
stack in local minimum --> break loop
res = 2.000000052981175 / seed = 3340220747
loss(mes) : 2.000000052981175


In [56]:
LH = loss_mes._transform([best_model.matrix()]*loss_mes._n_unitaries, original = True).detach().numpy()

In [87]:
L = 4
H = sum_ham(lh, [[i, (i+1)%L] for i in range(L)], L, 3)

In [88]:
np.linalg.eigvalsh(H)

array([-2.66666667, -2.00000005, -2.00000005, -1.99999996, -1.19419191,
       -1.19419191, -1.19419191, -1.19419191, -1.1941919 , -1.1941919 ,
       -1.0971676 , -1.0971676 , -1.09716753, -1.09716753, -1.09716751,
        0.66666659,  0.66666659,  0.6666666 ,  0.66666669,  1.33333313,
        1.33333313,  1.33333317,  1.33333317,  1.33333317,  1.33333317,
        1.33333317,  1.33333317,  1.33333328,  1.33333328,  1.33333331,
        1.33333331,  1.33333331,  1.33333331,  1.33333333,  1.33333333,
        1.33333333,  1.33333335,  1.33333335,  1.33333339,  1.33333339,
        1.3333334 ,  1.86085844,  1.86085844,  1.86085851,  1.86085851,
        1.86085851,  1.86085851,  1.99999991,  2.43050073,  2.43050073,
        2.43050079,  2.43050079,  2.43050082,  2.66666657,  2.66666657,
        2.66666659,  2.66666659,  2.66666659,  3.33333313,  3.33333313,
        3.33333313,  3.33333313,  3.33333324,  3.33333324,  3.33333324,
        3.33333324,  3.3333333 ,  3.3333333 ,  3.3333333 ,  3.33

In [92]:
np.linalg.eigvalsh(lh)

array([-1.33333337, -1.33333333, -1.33333333, -1.33333323, -1.33333323,
        0.66666665,  0.66666667,  0.66666667,  0.66666668])